In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, beta, gamma
from scipy.stats import multivariate_normal as mn
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
rawdata = pd.read_csv('gapminder.tsv.txt', sep='\t')
life_exp = rawdata['lifeExp']
continent = pd.get_dummies(rawdata['continent'],drop_first=True,prefix="continent")
continent+=1
data = rawdata[['year', 'country', 'pop', 'gdpPercap']]
data = pd.concat([data, continent], axis=1)

data

year      country       pop   gdpPercap  continent_Americas  \
0     1952  Afghanistan   8425333  779.445314                   1   
1     1957  Afghanistan   9240934  820.853030                   1   
2     1962  Afghanistan  10267083  853.100710                   1   
3     1967  Afghanistan  11537966  836.197138                   1   
4     1972  Afghanistan  13079460  739.981106                   1   
...    ...          ...       ...         ...                 ...   
1699  1987     Zimbabwe   9216418  706.157306                   1   
1700  1992     Zimbabwe  10704340  693.420786                   1   
1701  1997     Zimbabwe  11404948  792.449960                   1   
1702  2002     Zimbabwe  11926563  672.038623                   1   
1703  2007     Zimbabwe  12311143  469.709298                   1   

      continent_Asia  continent_Europe  continent_Oceania  
0                  2                 1                  1  
1                  2                 1                  1  
2                  2                 1                  1  
3                  2                 1                  1  
4                  2                 1                  1  
...              ...               ...                ...  
1699               1                 1                  1  
1700               1                 1                  1  
1701               1                 1                  1  
1702               1                 1                  1  
1703               1                 1                  1  

[1704 rows x 8 columns]

In [3]:
countries = rawdata['country'].unique()
betas = []
life_exps = []

for country in countries:
    dat = data[data['country']==country]
    dat=dat.drop(['country'], axis=1)
    dat = dat.reset_index()
    dat = dat.drop(['index'], axis=1)
    y = np.array(rawdata[rawdata['country']==country]['lifeExp'])
    life_exps.append(y)
    
    b = np.linalg.lstsq(dat, y, rcond=None)
    betas.append(b[0])

In [4]:
# create gibbs sampler
def gibbs(data, life_exp, beta0, N=1000, burn=100):
    beta_post = np.zeros(N-burn)
    
    # get all initial vals
    k = norm()